## Lab2: Performance Tuning


### Timing python code

In [ ]:
import timeit
import numpy as np
from math import log10 as lg10
import matplotlib.pyplot as plt

def f(x):
    return x * x

%timeit -n 10 f(3)

In [ ]:
mycode = ''' 
def f(x): 
    return x*x 
f(x_val)
'''

x_val = 3
total_time = timeit.timeit(mycode, number = 10, globals=globals())
print(f'{total_time/10} sec per loop')

### Data Structure Choice: Membership Testing

### List

- Searching for an element in list takes O(N)

In [ ]:
letters = 'ASDFGHJKLQWERTYUIOPZXCVBNM'

letters_list = [x + y + z for x in letters for y in letters for z in letters]

print(letters_list[:10])
# note that ABC is not the first element but AAA

%timeit -n 100 'ABC' in letters_list 

%timeit -n 100 'PQR' in letters_list

In [ ]:
import matplotlib.pyplot as plt

mycode = '''
def check_membership(elem):
    return elem in numbers_list
check_membership(i)
'''
times = []
for i in range(0,100000,10):
    numbers_list = np.random.randint(0, i, i)
    total_time = timeit.timeit(mycode, number = 5, globals=globals())
    
    times.append(total_time / 5)
    

In [ ]:
plt.plot(times)
plt.xlabel('Size of the list')
plt.ylabel('Lookup time')
plt.show()

### Dictionary
- Dictionaries are implemented as a Hash table, which hash the key
- _Dicts_ and _sets_ are fast when looking up elements. 
- Insert, search and delete operations are O(1)



In [ ]:
letters_dict = {x: x for x in letters_list}
# Time how long it takes to find ‘abc’ and 'pqr'in letters_dict.

print('in dict')
%timeit -n 100 'ABC' in letters_dict
%timeit -n 100 'PQR' in letters_dict

In [ ]:
mycode = '''
def check_membership(elem):
    return elem in numbers_dict
check_membership(i)
'''
times = []
for i in range(0,10000,10):
    numbers_list = np.random.randint(0,i,i)
    numbers_dict = {k:k for k in numbers_list}
    total_time = timeit.timeit(mycode, number = 10, globals=globals())
    
    times.append(total_time/10)
    

In [ ]:
plt.plot(times)
plt.xlabel('Size of the dictionary')
plt.ylabel('Lookup time')

### Function Choice: String Concatenation

- Python strings are immutable.
- str1 + str2 creates a new string.
- This copying can lead to significant slowdown

In [ ]:
def method1():
    out_str = ''
    global loop_count
    for num in range(loop_count):
        out_str += 'num'
    return out_str

def method2():
    str_list = []
    global loop_count
    for num in range(loop_count):
        str_list.append('num')
    return ''.join(str_list)

def method3():
    global loop_count
    return ''.join(['num' for i in range(loop_count)])


In [ ]:
loop_count = 100000

%timeit -n 10 method1()
%timeit -n 10 method2()
%timeit -n 10 method3()

### Optimizing loops

- Avoid for loops, use map or numpy operations
- Numpy is faster due to vectorized implementations

Multiply two 1000x1000 matrices

In [ ]:
N = 100
arr1 = np.random.random((N,N))
arr2 = np.random.random((N,N))

### How fast is  For loop ?

In [ ]:
%%timeit -n 1

def multiply(x,y):

    m1,n1 = x.shape
    m2,n2 = y.shape
    
    assert(n1 == m2)
    z = np.zeros((m1,n2))

    for i in range(m1): 
        for j in range(n2): 
            for k in range(m2): 
                z[i][j] += x[i][k] * y[k][j]
                
    return z

multiply(arr1, arr2)

### How fast is numpy?

In [ ]:
%%timeit -n 1

def mod_multiply(x,y):
    """
    Multiply two arrays using numpy.
    """
    return np.matmul(x,y)

mod_multiply(arr1, arr2)

### Decorators

In Python, functions are the first class objects, which means that:

- Functions are objects; they can be referenced to, passed to a variable and returned from other functions as well.

- Functions are taken as the argument into another function and then called inside the wrapper function.


In [ ]:
# defining a decorator, pass in "func", return "wrapper"

def my_decorator(func):
    
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)
say_whee()

In [ ]:
@my_decorator
def say_whee():
    print("Whee!")
say_whee()

### Caching with decorators

- Decorators can be used to cache intermediate values
- Can be used to avoid repeated calculations (dynamic programming)

In [ ]:
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)

def cache_fib(f): 
    memory = {} 
  
    # This inner function has access to memory 
    # and 'f' 
    def inner(num): 
        if num not in memory:          
            memory[num] = f(num) 
        return memory[num] 
  
    return inner 

@cache_fib
def better_fib(i):
    if i < 2: return 1
    return better_fib(i-1) + better_fib(i-2)


In [ ]:
%timeit -n 1 fib(30)


In [ ]:
%timeit -n 1 better_fib(30)

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=128)
def better_fib(i):
    if i < 2: return 1
    return better_fib(i-1) + better_fib(i-2)

%timeit -n 1 better_fib(30)